# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('churn').getOrCreate()

In [15]:
data = spark.read.csv('../customer_churn.csv',inferSchema=True,header=True)

In [16]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [17]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [18]:
# 필요한 컬럼만 가져옴
my_cols = data.select([ # 'Names',
 # 'Age', # client의 나이는 영향을 주지 않을 것으로 예상
 'Total_Purchase',
 # 'Account_Manager', 무작위로 배정되기 때문에 제외한다.
 'Years',
 'Num_Sites',
 'Onboard_date',
 # 'Location', 웹 광고인데 회사 위치는 중요하지 않을 것이다.
 # 'Company', 회사 이름은 영향을 끼치지 않을 것이다.
 'Churn'])

In [19]:
from pyspark.sql.functions import isnan, when, count, col, isnull

# 결측값 확인
my_cols.select([count(when(isnull(c), c)).alias(c) for c in my_cols.columns]).show()

+--------------+-----+---------+------------+-----+
|Total_Purchase|Years|Num_Sites|Onboard_date|Churn|
+--------------+-----+---------+------------+-----+
|             0|    0|        0|           0|    0|
+--------------+-----+---------+------------+-----+



In [20]:
my_cols.show()

+--------------+-----+---------+-------------------+-----+
|Total_Purchase|Years|Num_Sites|       Onboard_date|Churn|
+--------------+-----+---------+-------------------+-----+
|       11066.8| 7.22|      8.0|2013-08-30 07:00:40|    1|
|      11916.22|  6.5|     11.0|2013-08-13 00:38:46|    1|
|      12884.75| 6.67|     12.0|2016-06-29 06:20:07|    1|
|       8010.76| 6.71|     10.0|2014-04-22 12:43:12|    1|
|       9191.58| 5.56|      9.0|2016-01-19 15:31:15|    1|
|      10356.02| 5.12|      8.0|2009-03-03 23:13:37|    1|
|      11331.58| 5.23|     11.0|2016-12-05 03:35:43|    1|
|       9885.12| 6.92|      9.0|2006-03-09 14:50:20|    1|
|       14062.6| 5.46|     11.0|2011-09-29 05:47:23|    1|
|       8066.94| 7.11|     11.0|2006-03-28 15:42:45|    1|
|      11575.37| 5.22|      8.0|2016-11-13 13:13:01|    1|
|       8771.02| 6.64|     11.0|2015-05-28 12:14:03|    1|
|       8988.67| 4.84|     11.0|2011-02-16 08:10:47|    1|
|       8283.32|  5.1|     13.0|2012-11-22 05:35:03|    

In [21]:
# VectorAssembler가 timestamp 유형은 지원하지 않아서 date 유형으로 바꿈
# 근데 date 유형도 지원안함
# 빼고 진행한다.
# my_cols = my_cols.withColumn('trans_Onboard_date', my_cols['Onboard_date'].cast('date'))

문자열이 없어 인덱서나 인코딩할 필요없다.
굳이 파이프라인을 사용하지 않는다.

In [33]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Total_Purchase',
 'Years',
 'Num_Sites',
#  'trans_Onboard_date',],
    ],
    outputCol='features')

In [34]:
final_cols = assembler.transform(my_cols)

In [35]:
from pyspark.ml.classification import LogisticRegression

In [42]:
log_reg = LogisticRegression(featuresCol='features',labelCol='Churn')

In [45]:
train_data, test_data = final_cols.randomSplit([0.7,.3])

In [46]:
fit_log_reg = log_reg.fit(train_data)

In [48]:
results = fit_log_reg.transform(train_data)

In [49]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [50]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [51]:
AUC = my_eval.evaluate(results)
AUC

0.7320165719240774